<a href="https://colab.research.google.com/github/trefftzc/partition_COLAB_notebooks/blob/main/partition_mpi4py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving the partition problem with MPI
The library mpi4py allows a python developer to use MPI to solve a problem.

The documentation for mpi4py is available in this web site: https://mpi4py.readthedocs.io/en/stable/intro.html

mpi4py is not installed by default on COLAB, hence it is necessary to install it on COLAB.

In [1]:

!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.6 MB/s eta 0:00:00


We will write the source code to a python file that can be later executed using the ! option in COLAB.
The ! option allows one to access a command line.

This is very handy to be able to use input redirection with test files.

The general design of the program is to partition the range of values
that need to be tested among the participating nodes.
Every node works on a subset of the values.

Notice that MPI calls were needed only in the main function.
The  functions evaluatePartition and printResults did not require any changes.

The function solve now receives two additional parameters, the begin and
the end of the subrange of values being tested on this particular node.




In [15]:
%%writefile partition_mpi4py.py
from mpi4py import MPI
import time
import numpy as np

# This function has not changed from the previous versions
# There are no changes related to MPI in this function

def evaluatePartition(value,n,array):

  sum0s = 0
  sum1s = 0
  mask = 1
  for i in range(0,n):
    if ((mask & value) != 0):
      sum1s = sum1s + array[i]
    else:
      sum0s = sum0s + array[i]
    mask = mask*2

  if (sum0s == sum1s):
    return 1
  else:
    return 0

# This function has not changed from the previous versions
# There are no changes related to MPI in this function

def printResults(value,n,array):
  print("Solution:")
  print("First partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) != 0):
      print(array[i],end=" ")
      sum = sum + array[i]
    mask = mask * 2

  print("sum: ",sum)
  print("Second partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) == 0):
      print(array[i],end=" ")
      sum = sum + array[i]
    mask = mask * 2

  print("sum: ",sum)


def solve(numpyArray,nPartitions,n,begin,end):
  solutionFound = 0
  for i in range(begin,end):
    r = evaluatePartition(i,n,numpyArray)
    if (r==1):
      solutionFound = max(solutionFound,i)
  if (solutionFound != 0):
    return solutionFound
  else:
    return -1

# The changes necessary to use MPI (mpi4py)
# are limited to the main program

if __name__ == "__main__":

  comm = MPI.COMM_WORLD
  rank = comm.Get_rank()
  number_nodes = comm.Get_size()
  # If this is the master node, read the problem
  if rank == 0:
    start = time.time()
  if rank == 0:
    n = int(input())
    valuesString = input()
    values = valuesString.split()
    for i in range(len(values)):
      values[i] = int(values[i])
  # Print the instance of the problem
    print("Problem size: ",n)
    print("Problem instance: ",values)
  else:
    n = None

# Broadcast the size of the problem, the number of entries
# in the multiset to all participating nodes
  n = comm.bcast(n,root=0)

# Broadcast the multiset to all participating nodes
  if rank == 0:
    npValues = np.array(values)
  else:
    npValues = np.arange(n,dtype='i')

  npValues = comm.bcast(npValues,root=0)

# Calculate the size of the power set
  nPartitions = int ( 2 ** n )

# Calculate the portion of the power set that each
# node will be working on
  portionEachNode = nPartitions // number_nodes
  initial = portionEachNode * rank;
  if (rank != (number_nodes-1)):
    final = initial + portionEachNode
  else:
    final = nPartitions

  max_in_this_node = solve(npValues,nPartitions,n,initial,final)

  # result = comm.reduce(max_in_this_node,op=MPI.MAXLOC,root=0)
  result = comm.reduce(max_in_this_node,op=MPI.MAX,root=0)

  if rank == 0:
    if result == -1:
      print("No solution found.")
    else:
      printResults(result,n,npValues)
      print("The index for the result is: ",result)
  if rank == 0:
    end = time.time()
    elapsed = end - start
    print("The program took: ",elapsed," seconds.")



Overwriting partition_mpi4py.py


In [3]:
%%writefile test11.Text
11
1 2 3 4 5 6 7 8 9 10 11

Writing test11.Text


In [16]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python partition_mpi4py.py < test11.Text


Problem size:  11
Problem instance:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Solution:
First partition: 
3 9 10 11 sum:  33
Second partition: 
1 2 4 5 6 7 8 sum:  33
The index for the result is:  1796
The program took:  0.007276296615600586  seconds.


In [ ]:
%%writefile test12.Text
12
1 1 1 1 1 1 1 1 1 1 1 11

Writing test12.Text


In [ ]:
%%writefile test4.Text
4
1 1 1 3

Writing test4.Text


# And if we add NUMBA, do we get better performance?


In [ ]:
%%writefile partition_mpi4py_numba.py
from mpi4py import MPI
import time
import numpy as np
from numba import jit


# This function has not changed from the previous versions
# There are no changes related to MPI in this function
@jit(nopython=True)
def evaluatePartition(value,n,array):

  sum0s = 0
  sum1s = 0
  mask = 1
  for i in range(0,n):
    if ((mask & value) != 0):
      sum1s = sum1s + array[i]
    else:
      sum0s = sum0s + array[i]
    mask = mask*2

  if (sum0s == sum1s):
    return 1
  else:
    return 0

# This function has not changed from the previous versions
# There are no changes related to MPI in this function

def printResults(value,n,array):
  print("Solution:")
  print("First partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) != 0):
      print(array[i],end=" ")
      sum = sum + array[i]
    mask = mask * 2

  print("sum: ",sum)
  print("Second partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) == 0):
      print(array[i],end=" ")
      sum = sum + array[i]
    mask = mask * 2

  print("sum: ",sum)

@jit(nopython=True)
def solve(numpyArray,nPartitions,n,begin,end):
  solutionFound = 0
  for i in range(begin,end):
    r = evaluatePartition(i,n,numpyArray)
    if (r==1):
      solutionFound = max(solutionFound,i)
  if (solutionFound != 0):
    return solutionFound
  else:
    return -1

# The changes necessary to use MPI (mpi4py)
# are limited to the main program

if __name__ == "__main__":

  comm = MPI.COMM_WORLD
  rank = comm.Get_rank()
  number_nodes = comm.Get_size()
  # If this is the master node, read the problem
  if rank == 0:
    start = time.time()
  if rank == 0:
    n = int(input())
    valuesString = input()
    values = valuesString.split()
    for i in range(len(values)):
      values[i] = int(values[i])
  # Print the instance of the problem
    print("Problem size: ",n)
    print("Problem instance: ",values)
  else:
    n = None

# Broadcast the size of the problem, the number of entries
# in the multiset to all participating nodes
  n = comm.bcast(n,root=0)

# Broadcast the multiset to all participating nodes
  if rank == 0:
    npValues = np.array(values)
  else:
    npValues = np.arange(n,dtype='i')

  npValues = comm.bcast(npValues,root=0)

# Calculate the size of the power set
  nPartitions = int ( 2 ** n )

# Calculate the portion of the power set that each
# node will be working on
  portionEachNode = nPartitions // number_nodes
  initial = portionEachNode * rank;
  if (rank != (number_nodes-1)):
    final = initial + portionEachNode
  else:
    final = nPartitions

  max_in_this_node = solve(npValues,nPartitions,n,initial,final)

  result = comm.reduce(max_in_this_node,op=MPI.MAX)
  if rank == 0:
    if result == -1:
      print("No solution found.")
    else:
      printResults(result,n,npValues)
  if rank == 0:
    end = time.time()
    elapsed = end - start
    print("The program took: ",elapsed," seconds.")

Overwriting partition_mpi4py_numba.py


In [ ]:

%%writefile test20.Text
20
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20

Overwriting test20.Text


In [ ]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python partition_mpi4py.py < test20.Text

Problem size:  20
Problem instance:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Solution:
First partition: 
15 16 17 18 19 20 sum:  105
Second partition: 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 sum:  105
The program took:  7.852737665176392  seconds.


In [ ]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python partition_mpi4py_numba.py < test20.Text

Problem size:  20
Problem instance:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Solution:
First partition: 
15 16 17 18 19 20 sum:  105
Second partition: 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 sum:  105
The program took:  1.0843636989593506  seconds.
